# Validating Your Data

# Figuring out what's in your Data

In [2]:
from lxml import objectify
import pandas as pd


xml = objectify.parse(open('XMLData2.xml'))
root = xml.getroot()
df = pd.DataFrame(columns=('Number', 'String', 'Boolean'))

for i in range(0,4):
    obj = root.getchildren()[i].getchildren()
    row = dict(zip(['Number', 'String', 'Boolean'],
                   [obj[0].text, obj[1].text,
                   obj[2].text]))
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)
    
    
    search = pd.DataFrame.duplicated(df)
    print(df)
    print()
    print(search[search == True])

  Number String Boolean
0      1  First    True

Series([], dtype: bool)
  Number  String Boolean
0      1   First    True
1      2  Second   False

Series([], dtype: bool)
  Number  String Boolean
0      1   First    True
1      2  Second   False
2      3   Third    True

Series([], dtype: bool)
  Number  String Boolean
0      1   First    True
1      2  Second   False
2      3   Third    True
3      3   Third    True

3    True
dtype: bool


# Removing Duplicate

In [3]:
from lxml import objectify
import pandas as pd

xml = objectify.parse(open('XMLData2.xml'))
root = xml.getroot()
df = pd.DataFrame(columns=('Number', 'String', 'Boolean'))
for i in range(0,4):
    obj = root.getchildren()[i].getchildren()
    row = dict(zip(['Number', 'String', 'Boolean'],
                  [obj[0].text, obj[1].text, obj[2].text]))
    row_s = pd.Series(row)
    row_s.name = i
    df = df.append(row_s)
    
print(df.drop_duplicates())

  Number  String Boolean
0      1   First    True
1      2  Second   False
2      3   Third    True


# Creating a Data Map and Data Plan

In [4]:
import pandas as pd
pd.set_option('display.width', 55)


df = pd.DataFrame({'A': [0,0,0,0,0,1,1],
                   'B': [1,2,3,5,4,2,5],
                   'C': [5,3,4,1,1,2,3]})

a_group_desc = df.groupby('A').describe()
print(a_group_desc)

      B                                            \
  count mean       std  min   25%  50%   75%  max   
A                                                   
0   5.0  3.0  1.581139  1.0  2.00  3.0  4.00  5.0   
1   2.0  3.5  2.121320  2.0  2.75  3.5  4.25  5.0   

      C                                            
  count mean       std  min   25%  50%   75%  max  
A                                                  
0   5.0  2.8  1.788854  1.0  1.00  3.0  4.00  5.0  
1   2.0  2.5  0.707107  2.0  2.25  2.5  2.75  3.0  


Using the stacked version on this data set 

In [5]:
stacked = a_group_desc.stack()
print(stacked)

                B         C
A                          
0 count  5.000000  5.000000
  mean   3.000000  2.800000
  std    1.581139  1.788854
  min    1.000000  1.000000
  25%    2.000000  1.000000
  50%    3.000000  3.000000
  75%    4.000000  4.000000
  max    5.000000  5.000000
1 count  2.000000  2.000000
  mean   3.500000  2.500000
  std    2.121320  0.707107
  min    2.000000  2.000000
  25%    2.750000  2.250000
  50%    3.500000  2.500000
  75%    4.250000  2.750000
  max    5.000000  3.000000


To see only the number of items in each series and their mean. Use this code to reduce the size of the information output

In [6]:
print(a_group_desc.loc[:,(slice(None),['count','mean']),])

      B          C     
  count mean count mean
A                      
0   5.0  3.0   5.0  2.8
1   2.0  3.5   2.0  2.5


# Creating categorical variables

Categorical variables have specific number of values, which makes them incredibly valuable in performing a number of data science tasks. For example, imagine trying to find value that are out of range in a huge dataset. In this example, you see one method for creating a categorical variable and then using it to check whether some data falls within the specified limits:

In [7]:
import pandas as pd

car_colors = pd.Series(['Blue', ' Red', 'Green'],
                      dtype='category')

car_data = pd.Series(
    pd.Categorical(
        ['Yellow', 'Green', 'Red', 'Blue', 'Purple'],
                 categories=car_colors, ordered=False))


find_entries = pd.isnull(car_data)

print(car_colors)
print()
print(car_data)
print()


0     Blue
1      Red
2    Green
dtype: category
Categories (3, object): [Red, Blue, Green]

0      NaN
1    Green
2      NaN
3     Blue
4      NaN
dtype: category
Categories (3, object): [Red, Blue, Green]



The example begins by creating a categorical variable, car_colors. The variable contains the value Blue, Red and Green as colors that are acceptable for a car. Notice that you must specify a dtype property value of category. The next step is to create another series. This one uses a list of actual car colors, named car_data, as imput. Not all the car colors match the predefined acceptablevalues. When this problem occurs, pandas output Not a Number(NaN) instead of the car color.


Of course , you could search the list manually for the nonconforming cars, but the easiest method is to have pandas do the work for you. In this case, you ask pandas which entries are null using isnull() and place them in find_entries. You can see from the output of the code above.

Looking at the list of car_data outputs, you can see thats entries 0 and 4 equal to NaN. The output from find_entries verifies this fact for you. If this were a large dataset, you could quickly locate and correct errant entries in the data set before performing an analysis on it

# Renaming levels

There are times when the naming of categories you use is inconvinient or otherwise wrong for a particular need as needed using the technique shown in the following example.

In [8]:
import pandas as pd

car_colors = pd.Series(['Blue', 'Red', 'Green'], 
                       dtype='category')
car_data = pd.Series(
    pd.Categorical(
        ['Blue', 'Green', 'Red', 'Blue', 'Red'],
        categories=car_colors, ordered=False))


car_colors.cat.categories = ["Purple", "Yellow", "Mauve"]
car_data.cat.categories = car_colors

print(car_data)

0    Purple
1    Yellow
2     Mauve
3    Purple
4     Mauve
dtype: category
Categories (3, object): [Purple, Yellow, Mauve]


All you need to do is set cat.categories property to a new value, as shown.

# Combining Levels

A particular categorical level might be too small to offer significant data for analysis. Perhaps there are only a few of the values, which may not be enoughto create a statistical difference. In this case, combining several small categories might offer better  analysis result. The following exemple shows how to combine categories : 

In [9]:
import pandas as pd

car_colors = pd.Series(['Blue', 'Red', 'Green'], 
                       dtype='category')
car_data = pd.Series(
    pd.Categorical(
        ['Blue', 'Green', 'Red', 'Green', 'Red', 'Green'],
        categories=car_colors, ordered=False))

car_data.cat.categories = ["Blue_Red", "Red", "Green"]
print (car_data[car_data.isin(['Red'])])

car_data[car_data.isin(['Red'])] = 'Blue_Red'

car_data.loc[car_data.isin(['Red'])] = 'Blue_Red'
car_data.loc[car_data.isin(['Blue'])] = 'Blue_Red'
car_data = car_data.cat.set_categories(
    ["Green", "Blue_Red"])

print()
print(car_data)




1    Red
3    Red
5    Red
dtype: category
Categories (3, object): [Blue_Red, Red, Green]

0    Blue_Red
1    Blue_Red
2       Green
3    Blue_Red
4       Green
5    Blue_Red
dtype: category
Categories (2, object): [Green, Blue_Red]


What this example shows you is that there is only one Blue item and only two Red item, but there are three Green items, which places Green in the majority. Combining Blue and Red together is a two-step process. First, you add the Blue_Red category to car_data. Then you change the Red and Blue entries to Blue_Red, which create the combined category. As a final step, you can remove the unneeded categories.

However, before you can change the Red entries to Blue_Red entries, you must find them. This is where a combination of calls to isin(), which locate the Red entries, and loc[], which obtains their index, provides precisely what you need. The First print() statement show the result of this combination.

Notice that there are now three Blue_Red entries and three Green entries. The result is thath the levels are now combined as expected

# Dealing with Date In Your Data

# Formatting date and time values

Obtaining the correct date and time representation can make performing analysis a lot easier. For example, you ofter have to change the representation to obtain a correct sorting of values. Python provides two common method of formatting date and time. The first is to call str(), which simply turns a datetime value into a string without any formatting. The strftime(), you must provide a string containing special directives that define the formatting. You can find a listing of these directive at http://strftime.org/

Now that you have some idea of how time and date conversion work, it's time to see an exemple create datetime object and them converts it into a string using two different approaches:

In [10]:
import datetime as dt

now = dt.datetime.now()

print(str(now))
print(now.strftime('%a, %d, %B, %Y'))

2021-02-16 13:27:25.974477
Tue, 16, February, 2021


In this case, you can see that using str() is the easiest approach. However, as shown by the output, it may not provide the output you need. Using strftime() is infinitely more flexible.

# Using the right time transformation

Time zone and differences in local time can cause all sorts of problems when performing analysis. For that matter, some types of calculation simply require a time shift in order to get the right results. No matter what the reason, you may need to transform one time into another time at some point. The following example  show some technique you can employ to perform the task :

In [11]:
import datetime as dt

now = dt.datetime.now()
timevalue = now + dt.timedelta(hours=2)

print(now.strftime('%H:%M:%S'))
print(timevalue.strftime('%H:%M:%S'))
print(timevalue-now)


13:27:27
15:27:27
2:00:00


The timedelta() function makes time transformation straightfoward. You can use any of these parameter names with timedelta() to change a time and date value.
You can also manipulate time by performing additions or substraction on time values. You can even substract two time value to determinate the difference betwenn them Check the outpout of the code above.

Note that now is the local time , timevalue is two time zone different from this one, and there is a two-hour difference between the two times. You can perform all sorts of transformations using these technique to ensure that your analysis always showss precisely the time-oriented values you need.

# Dealing with missing Data

Sometimes the data you receive is missing information in specific fields. For example, a customer record might be missing an age. If enough recorsds are missing entries, any analysis you perform will be skewed and the result of the analysis weighted in a unpredictable manner. Having a strategy for dealing with missing data is important. The following sections give some idea on how to work through these issue and produce better result.

# Finding the missing data 

Finding missing data in your dataset is essential to avoid getting incorrect results from analysis. The following code shows how you can obtain a listing of missing values without too much effort:

In [12]:
import pandas as pd
import numpy as np

s = pd.Series([1, 2, np.NaN, 5, 6, None, 'Hello', 0])

print(s.isnull())


print()
print(s[s.isnull()])

0    False
1    False
2     True
3    False
4    False
5     True
6    False
7    False
dtype: bool

2     NaN
5    None
dtype: object


A dataset can represent missing data in several ways. In this example, you see missing data represented as np.NaN (Numpy Not a Number) and the python None value.


Use the isnull() method to detect the missing values. The ouput of shows True when the value is missing. By adding an index into the dataset, you obtain just the entries that are missing. The example output above show the result of this method. 

# Encoding missingness

After you figured out that your dataset is missing information, you need to consider what do to about it. Three possibilities are to ignore the issue, fill in the missing items, or remove (drop) the missing entries from the dataset. Ignoring the issue could lead to all sorts of problems for your analysis, so it's the option you will use least often. The following example shows one technique for filling in missing data or dropping the errant entries from the datasset:

In [13]:
import pandas as pd
import numpy as np 

s = pd.Series([1, 2, 3, np.NaN, 5, 6, None])
print(s.fillna(int(s.mean())))
print()
print(s.dropna())

0    1.0
1    2.0
2    3.0
3    3.0
4    5.0
5    6.0
6    3.0
dtype: float64

0    1.0
1    2.0
2    3.0
4    5.0
5    6.0
dtype: float64


The two method of interest are fillna(), wich fill in the missing entries, dropna(), which drop the missing entries. When using fillna(), you must provide a value to use for the missing data. This example use the mean of all the values, but you could choose a number of other approaches. dropna() will remove the missing value as shown in the output above.

Technical Stuff : Working with a series is straightfoward because the dataset is so simple. When working with DATAFRAME , however, the problem becomes more significantly more complicated. You still have the option of dropping the entire row. When a column is sparsely populated, you might drop the column insted. Filling in the data also becomes more complex because you must consider the dataset as a whole, in addition to the needs of the individual feature.

# Imputing Missing Data 

The previous section hints at the processs of imputing missing data (ascribing characteristics based on how the data is used). The technique you use depends on the sort of data you're working with. For example, when working with a tree ensemble you may simply replace missing value with -1 and rely on the imputer ( a transformer algorithm used to complete misssing value) to define the best possible value for the missing data. The following example shows a ttechnique you can use to impute missing data values : 

In [14]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

s = [[1, 2, 3, np.NaN, 5, 6, None]]

imp = SimpleImputer(missing_values= np.NAN, strategy= 'mean', fill_value=None, verbose=0, copy=True)

imp.fit([[1, 2, 3, 4, 5, 6, 7]])

x = pd.Series(imp.transform(s).tolist()[0])

print(x)

0    1.0
1    2.0
2    3.0
3    4.0
4    5.0
5    6.0
6    7.0
dtype: float64


In this example, s is missing some values. The code create an SimpleImputer to replace these missing value. The missing_value parameter defines what to look for, which is np.NaN. you set the fill_value parameter to None to impute along the columns .

mean : Replace the value by using the mean along the axis

median : Replace the value by using the medium along the axis

most_frequent: Replaces the value by using the most frequent value along the axis

Before you can impute anything, you must provide statistic for the SimpleImputer to use by calling fit(). The code then calls transform() on s to fill in the missing values. In this case, the example needs to display the result as a series. To create a series, you must convert the SimpleImputer output to a list and use the resulting list as input to Series()

# Slicing and Dicing : Filtering and Selecting Data

You may not need to work with all the data in a dataset. In fact, looking at just one particular column might be benificial, such as age, or a set of rows with a significant amount of information. You perform two steps to obtain just the data you need to perform a particular task :

1. Filter rows to create a subject of the data that meets the criterion you select
   (such as the people between the ages of 5 and 10)

2. Select data columns that contain the date you need to analyze. For example you probably don't need the individual's names unless you want to perform some analysisbased of name.

 The act of slicing and dicing data, gives you a subset of the data suitable for analysis. The following sections describe various ways to obtain specific pieces of data to meet particular needs.

# Slicing rows 

Slicing can occur in multiple when working with data, but the technique of interest in this section is to slice from row of 2-D or 3-D data. A 2-D array may contain temperatures ( x axis) over a specific time frame (y axis). Slicing a row would mean seeing temperaturres at a specific time. In some cases, you might associate rows with cases in a dataset. 

A 3-D array might include an axis for place (x axis), product (Y axis), and time (z axis) so that you can see sales for items over time. Perhaps you want to track whether the sales of an item are increasing, and specifically where they are increasing. Slicing a row would mean would mean seeing all the sales for one specific product for all locations at any time. The following example demonstrates how to perform this task:

In [15]:
x = np.array ([[[1, 2, 3], [4, 5, 6], [7, 8, 9],],
              [[11,12,13], [14,15,16], [17,18,19],],
              [[21,22,23], [24,25,26], [27,28,29]]])

x[1]

array([[11, 12, 13],
       [14, 15, 16],
       [17, 18, 19]])

In this case, the example builds a 3-D array. It then slice row 1 of that array to produce the above output

# Slicing columns 


Using the examples from previous sections, slicing columns would obtain data at a 90- degree angle from rows. In other words, when working with the 2-D array, you would want to see the times at which specific temperaturs occured.
Likewise, you might want to see the sales of all product for a specific location at any time when working with the 3-D array. In some cases, you might associate columns with features in a dataset. The following exaple demonstrates how to perform this task using the same array as in the previous section:

In [16]:
x = np.array ([[[1, 2, 3], [4, 5, 6], [7, 8, 9],],
              [[11,12,13], [14,15,16], [17,18,19],],
              [[21,22,23], [24,25,26], [27,28,29]]])

x[:,1]

array([[ 4,  5,  6],
       [14, 15, 16],
       [24, 25, 26]])

Note that the indexing now occurs at two levels. The first index refers to the row. Using the colon (:) for the row means to use all the rows. The second index refers to a column. In this case, the output will contain column 1. As seen in the ouput above.

Remember : This is a 3-D array. Therefore, each of the columns contains all the z axis elements. What you see is every row -  0 through 2 for that column with every z axis element 0 through 2 for that column

# Dicing

The act of dicing means to perform both row and column slicing such that you end up with a data wedge. For example, when working with the 3-D array, you might want to see the sales of a specific product in a specific location at any time. The following example demostrates how to perform this task using the same array as in the previous two sections:

In [17]:
x = np.array ([[[1, 2, 3], [4, 5, 6], [7, 8, 9],],
              [[11,12,13], [14,15,16], [17,18,19],],
              [[21,22,23], [24,25,26], [27,28,29]]])

print(x[1,1])
print(x[:,1,1])
print(x[1, :, 1])
print()
print(x[1:2, 1:2])


[14 15 16]
[ 5 15 25]
[12 15 18]

[[[14 15 16]]]


This example dices the array in four different ways. First you get row 1, column 1.
Of course, what you may actually want is column 1, z axis 1. If thats not quite right, you could always request row 1, z axis 1 instead. then again, you maywant row 1 and 2 of column 1 and 2. Check the output above for all request.

# Concatenating and Transforming

Data used for data science purposes seldom comes in a neat package. You may need to work with multiple databases in various locations -- each of which has its own data format. it's impossible to perform analysis on such disparate sources of information with any accuracy. To make the data useful, you must create a single dataset (by concatenating, or combining, the data from various source).

 Part of the process is to ensure that each field you create for the combined dataset has the same characteristics. For example, an age field in one database might appear as a string, but another database could use an integer for the same field. For the field to work together, they must appear as the same type of information.

The following section help you understand the process involve in concatenating and transforming data from various sources to create a single dataset. After you have a single dataset from these sources, you can begin to perform task such as analysis on the data. Of course, the trick is to create a single dataset that truly represents the data in all those disparate datasets -- modifying the data would result in skewed result.

# Adding new cases and variable

You often find a need to combine dataset in various ways or even to add new information for the sake of analysis purposes. The result is a combined dataset that includes either new cases or variables. The following example show techniques for performing both tasks:

In [26]:
import pandas as pd

df = pd.DataFrame({'A': [2,3,1],
                   'B': [1,2,3],
                   'C': [5,3,4]})

df1 = pd.DataFrame({'A': [4],
                    'B': [4],
                    'C': [4]})


df = df.append(df1)
df = df.reset_index(drop=True)
print(df)


df.loc[df.last_valid_index() + 1] = [5, 5, 5]
print()
print(df)

df2 = pd.DataFrame({'D': [1, 2, 3, 4, 5]})

df = pd.DataFrame.join(df, df2)
print()
print(df)

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4
3  4  4  4

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4
3  4  4  4
4  5  5  5

   A  B  C  D
0  2  1  5  1
1  3  2  3  2
2  1  3  4  3
3  4  4  4  4
4  5  5  5  5


The easiest way to add more data to an existing DataFrame is to rely on the append() method. you can also use the concat() method. In this case, the three cases found in df are added to single case found in df1. To ensure that the data is appended as anticipated, the columns in df and df1 must match. When you append two DataFrame objects in this manner, the new DataFrame contains the old index values. Use the reset_index() method to create a new index to make accessing cases easier.

You can also add another case to an existing DataFrame by creating the new case directly. Any time you add a new entry at a position that is one greater than the last_valid_index(), you get a new case result.

Sometimes you need to add a new variable (column) to the DataFrame. In this case, you rely on join() to perform the task. The resulting DataFrame will match cases with the same index value, so indexing is important. In addition, unless you want blank value, the number of cases in both DataFrame objects must match. Take a look at the output above for examples.

# Removing data

At some point, you may need to remove cases or variables from a dataset because they aren't requires for your analysis. In both cases, you rely on the drop() method to perform the task. The difference in removing cases or variables is in how you describe what to remove, as shown in the following example :

In [32]:
import pandas as pd

df = pd.DataFrame({'A': [2,3,1],
                   'B': [1,2,3],
                   'C': [5,3,4]})
print(df)

df = df.drop(df.index[[1]])
print()
print(df)


df = df.drop('B', 1)
print()
print(df)

   A  B  C
0  2  1  5
1  3  2  3
2  1  3  4

   A  B  C
0  2  1  5
2  1  3  4

   A  C
0  2  5
2  1  4


The example begins by removing a case from df. Notice how the code relies on an index to describe what to remove. You can  remove just one case(as shown), ranges of cases, or individual cases separated by commas. The main concern is to ensure that you have the correct index numbers for the cases you want to remove.

Removing a column is different. This example shows how to remove a column using a column name. You can also remove a column by using an index. In both cases, you must specify an axis as part of the removal process ( normally 1 ). Take a look at the output above. 

# Sorting and Shuffling

Sorting and shuffling are two ends of the same goal -- to manage data order. In the first case, you put the data in order, while in the second, you remove any systematic patterning from the order. In general, you don't sort datasets for the purpose of analysis because doing so can cause you to get incorrect results. However, you might want to sort data for presentation purposes. The following example code shows both Sorting and Shuffling:

In [34]:
import pandas as pd
import numpy as np

df = pd.DataFrame({'A': [2,1,2,3,3,5,4],
                   'B': [1,2,3,5,4,2,5],
                   'C': [5,3,4,1,1,2,3]})
print(df)

df = df.sort_values(by=['A', 'B'], ascending=[True, True])
df = df.reset_index(drop=True)
print()
print(df)


index = df.index.tolist()
np.random.shuffle(index)
df = df.loc[df.index[index]]
df = df.reset_index(drop=True)
print()
print(df)

   A  B  C
0  2  1  5
1  1  2  3
2  2  3  4
3  3  5  1
4  3  4  1
5  5  2  2
6  4  5  3

   A  B  C
0  1  2  3
1  2  1  5
2  2  3  4
3  3  4  1
4  3  5  1
5  4  5  3
6  5  2  2

   A  B  C
0  2  1  5
1  4  5  3
2  3  5  1
3  5  2  2
4  1  2  3
5  2  3  4
6  3  4  1


it turns out that sorting the data is a bit easier than shuffling it. To sort the data, you use the sort_value() method and define which columns to use for indexing purposes. You can also determine whether the index is ascending or descending order. Make sure to always call reset_index() when you're done so that the index appears in order for analysis or other purposes.

To shuffle the data, you first acquire the current index using df.index.tolist() and place index. A call to random_shuffle() creates a new order for the index. You then apply the new order to df using loc[]. As always, you call reset_index() to finalize the new order. Take a look at the output above.

# Aggregating Data at Any Level

Aggregation is the process of combining or gruping data together into a set, bag, or list. The data may not be alike. However, in most cases, an aggregation function combines several rows together statiscally using algorthms such as average, count, maximum, median, minimum, mode, or sum. There are several reason to aggregate data:


1. Make it easier to analyze
2. Reduce the ability of anyone to deduce the data of an individual from the dataset for privacy or other reasons
3. Create a combined data element from one data element from one data source that matches a combined data element in another source



The most important use of data aggregation is to promote anonymity in order to meet legal or other concerns. Sometimes even data that should be anonymous turns out to provide identification of an individual using the proper analysis techniques. For example, researchers have found that it's impossible to identify individuals based on just three credit card purchases ( see https://www.computerworld.com/article/2877935/how-three-small-credit-card-transactions-could-reveal-your-identity.html for details). Here's an example that shows how to perform aggreagation task :

In [36]:
import pandas as pd 



df = pd.DataFrame({'Map': [0,0,0,1,1,2,2],
                   'Values': [1,2,3,5,4,2,5]})

df['S'] = df.groupby ('Map')['Values'].transform(np.sum)
df['M'] = df.groupby ('Map')['Values'].transform(np.mean)
df['V'] = df.groupby ('Map')['Values'].transform(np.var)

print(df)

   Map  Values  S    M    V
0    0       1  6  2.0  1.0
1    0       2  6  2.0  1.0
2    0       3  6  2.0  1.0
3    1       5  9  4.5  0.5
4    1       4  9  4.5  0.5
5    2       2  7  3.5  4.5
6    2       5  7  3.5  4.5


In this case, you have two initial features for this DataFrame. The values in Map define which element in Values belong together. For example, when calculating a sum for Map index 0, you use the Values 1, 2, and 3.




To perform the aggregation, you must first call groupby() to group the Map values. You then index into Values and rely on transfor() to create the aggregated data using one of several algorithms found in NumPy, such as np.sum. Look at the output above.